# ساخت با مدل‌های Mistral

## مقدمه

این درس شامل موارد زیر است:  
- بررسی مدل‌های مختلف Mistral  
- درک موارد استفاده و سناریوهای هر مدل  
- نمونه‌های کد که ویژگی‌های منحصر به فرد هر مدل را نشان می‌دهند.


## مدل‌های میسترال

در این درس، ما سه مدل مختلف میسترال را بررسی خواهیم کرد:  
**میسترال بزرگ**، **میسترال کوچک** و **میسترال نِمو**.

هر یک از این مدل‌ها به صورت رایگان در بازار مدل‌های گیت‌هاب در دسترس هستند. کد این دفترچه از این مدل‌ها برای اجرای کد استفاده خواهد کرد. در اینجا جزئیات بیشتری درباره استفاده از مدل‌های گیت‌هاب برای [نمونه‌سازی با مدل‌های هوش مصنوعی](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) آمده است.


## Mistral Large 2 (2407)
Mistral Large 2 در حال حاضر مدل پرچمدار شرکت Mistral است و برای استفاده سازمانی طراحی شده است.

این مدل یک ارتقاء نسبت به Mistral Large اصلی است که ارائه می‌دهد
- پنجره متن بزرگ‌تر - ۱۲۸ هزار در مقابل ۳۲ هزار
- عملکرد بهتر در وظایف ریاضی و برنامه‌نویسی - دقت متوسط ۷۶.۹٪ در مقابل ۶۰.۴٪
- افزایش عملکرد چندزبانه - زبان‌ها شامل: انگلیسی، فرانسوی، آلمانی، اسپانیایی، ایتالیایی، پرتغالی، هلندی، روسی، چینی، ژاپنی، کره‌ای، عربی و هندی.

با این ویژگی‌ها، Mistral Large در موارد زیر برجسته است
- *تولید تقویت‌شده با بازیابی (RAG)* - به دلیل پنجره متن بزرگ‌تر
- *فراخوانی تابع* - این مدل دارای فراخوانی تابع بومی است که امکان ادغام با ابزارها و APIهای خارجی را فراهم می‌کند. این فراخوانی‌ها می‌توانند به صورت موازی یا به ترتیب پشت سر هم انجام شوند.
- *تولید کد* - این مدل در تولید کدهای Python، Java، TypeScript و C++ عملکرد برجسته‌ای دارد.


### مثال RAG با استفاده از Mistral Large 2


در این مثال، ما از Mistral Large 2 برای اجرای الگوی RAG بر روی یک سند متنی استفاده می‌کنیم. سوال به زبان کره‌ای نوشته شده و درباره فعالیت‌های نویسنده قبل از دانشگاه پرسیده است.

از مدل تعبیه‌های Cohere برای ایجاد تعبیه‌های سند متنی و همچنین سوال استفاده می‌کند. برای این نمونه، از بسته پایتون faiss به عنوان فروشگاه برداری استفاده می‌شود.

پرومپتی که به مدل Mistral ارسال می‌شود شامل هم سوالات و هم بخش‌های بازیابی شده‌ای است که به سوال شباهت دارند. سپس مدل پاسخ به زبان طبیعی ارائه می‌دهد.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## میسترال کوچک  
میسترال کوچک مدل دیگری از خانواده مدل‌های میسترال در دسته پرمیوم/سازمانی است. همانطور که از نامش پیداست، این مدل یک مدل زبان کوچک (SLM) است. مزایای استفاده از میسترال کوچک عبارتند از:  
- صرفه‌جویی در هزینه نسبت به مدل‌های بزرگ میسترال مانند میسترال بزرگ و NeMo - کاهش قیمت ۸۰٪  
- تأخیر کم - پاسخ سریع‌تر نسبت به مدل‌های بزرگ میسترال  
- انعطاف‌پذیر - قابلیت استقرار در محیط‌های مختلف با محدودیت‌های کمتر در منابع مورد نیاز.  

میسترال کوچک برای موارد زیر عالی است:  
- وظایف مبتنی بر متن مانند خلاصه‌سازی، تحلیل احساسات و ترجمه.  
- برنامه‌هایی که درخواست‌های مکرر دارند به دلیل صرفه‌جویی در هزینه  
- وظایف کد با تأخیر کم مانند بازبینی و پیشنهادات کد


## مقایسه میسترال کوچک و میسترال بزرگ

برای نشان دادن تفاوت در تأخیر بین میسترال کوچک و بزرگ، سلول‌های زیر را اجرا کنید.

شما باید تفاوتی در زمان پاسخ بین ۳ تا ۵ ثانیه مشاهده کنید. همچنین طول و سبک پاسخ‌ها را برای همان درخواست توجه کنید.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

در مقایسه با دو مدل دیگر که در این درس بررسی شدند، Mistral NeMo تنها مدل رایگان با مجوز Apache2 است.

این مدل به عنوان ارتقاء مدل متن‌باز قبلی Mistral، یعنی Mistral 7B، دیده می‌شود.

برخی ویژگی‌های دیگر مدل NeMo عبارتند از:

- *توکنیزاسیون کارآمدتر:* این مدل از توکنیزر Tekken به جای توکنیزر رایج‌تر tiktoken استفاده می‌کند. این امکان را فراهم می‌کند که عملکرد بهتری در زبان‌ها و کدهای بیشتر داشته باشد.

- *تنظیم دقیق (Finetuning):* مدل پایه برای تنظیم دقیق در دسترس است. این امکان انعطاف‌پذیری بیشتری برای موارد استفاده‌ای که نیاز به تنظیم دقیق دارند فراهم می‌کند.

- *فراخوانی توابع بومی* - مانند Mistral Large، این مدل بر روی فراخوانی توابع آموزش دیده است. این ویژگی آن را به یکی از اولین مدل‌های متن‌باز با این قابلیت تبدیل می‌کند.


## Mistral NeMo

در مقایسه با دو مدل دیگر که در این درس بررسی شدند، Mistral NeMo تنها مدل رایگان با مجوز Apache2 است.

این مدل به عنوان ارتقاء مدل متن‌باز قبلی Mistral، یعنی Mistral 7B، دیده می‌شود.

برخی ویژگی‌های دیگر مدل NeMo عبارتند از:

- *توکنیزاسیون کارآمدتر:* این مدل از توکنیزر Tekken به جای توکنیزر رایج‌تر tiktoken استفاده می‌کند. این امکان را فراهم می‌کند که عملکرد بهتری در زبان‌ها و کدهای بیشتر داشته باشد.

- *تنظیم دقیق (Finetuning):* مدل پایه برای تنظیم دقیق در دسترس است. این امکان انعطاف‌پذیری بیشتری برای موارد استفاده‌ای که نیاز به تنظیم دقیق دارند فراهم می‌کند.

- *فراخوانی توابع بومی* - مانند Mistral Large، این مدل بر روی فراخوانی توابع آموزش دیده است. این ویژگی آن را به یکی از اولین مدل‌های متن‌باز با این قابلیت تبدیل می‌کند.


### مقایسه توکن‌سازها

در این نمونه، نگاهی می‌اندازیم به اینکه Mistral NeMo چگونه توکنیزه کردن را در مقایسه با Mistral Large انجام می‌دهد.

هر دو نمونه همان پرامپت را می‌گیرند اما باید ببینید که NeMo تعداد توکن‌های کمتری نسبت به Mistral Large برمی‌گرداند.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## یادگیری در اینجا متوقف نمی‌شود، سفر را ادامه دهید

پس از اتمام این درس، مجموعه [یادگیری هوش مصنوعی مولد](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ما را بررسی کنید تا دانش خود در زمینه هوش مصنوعی مولد را ارتقا دهید!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما در تلاش برای دقت هستیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی خطاها یا نواقصی باشند. سند اصلی به زبان بومی خود باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچ گونه سوءتفاهم یا تفسیر نادرستی که از استفاده این ترجمه ناشی شود، نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
